In [1]:
import pickle
import pandas as pd
from explainers.dce import DistributionalCounterfactualExplainer
import os

In [2]:
data_path = 'data/hotel_booking'

In [3]:
factual = pd.read_csv(os.path.join(data_path,'factual.csv'))
counterfactual = pd.read_csv(os.path.join(data_path,'counterfactual.csv'))

In [4]:
factual

,lead_time,booking_changes,is_canceled
0,5,0,0.196734
1,15,3,0.004002
2,102,0,0.460798
3,336,0,0.676030
4,304,0,0.655259
5,44,0,0.377946
6,249,0,0.614473
7,31,0,0.317266
8,35,0,0.338983
9,160,0,0.540070


In [5]:
counterfactual

,lead_time,booking_changes,is_canceled
0,-27.675576,0.082796,0.099768
1,16.420959,3.003269,0.004049
2,170.649723,-0.049722,0.566292
3,339.902986,-0.050915,0.694042
4,309.304339,-0.053154,0.674890
5,-21.122848,0.144486,0.105640
6,283.514259,-0.281956,0.680484
7,-22.908241,0.130589,0.103603
8,48.835124,-0.052905,0.398829
9,78.794604,0.111577,0.397416


In [6]:
factual['is_canceled'].mean()

0.41815602205999997

In [7]:
counterfactual['is_canceled'].mean()

0.3725012649

In [15]:
factual.booking_changes.mean()

0.3

In [14]:
counterfactual.booking_changes.mean()

0.2984064805595042